In [4]:
import pickle
from pathlib import Path
import sys
sys.path.append("/g/g13/jones289/workspace/hd-cuda-master/hdpy")
from hdpy.hd_model import HDModel
from hdpy.prot_lig.aa_count_morgan_fp import PDBBindHD

In [5]:
proto_p = Path("/usr/WS1/jones289/hd-cuda-master/hdpy/hdpy/prot_lig/prototype_data.pkl")

with open(proto_p, 'rb') as handle:
    data = pickle.load(handle)
    
result_p = Path("/usr/WS1/jones289/hd-cuda-master/hdpy/hdpy/prot_lig/result_dict.pkl")

with open(result_p, 'rb') as handle:
    result_data = pickle.load(handle)


EOFError: Ran out of input

In [ ]:
data.keys()

In [ ]:
data['split_dict']['core_test'].keys()

In [ ]:
import numpy as np

split = "core_test"
# split = "refined_train"
test_hvs = np.concatenate([x.numpy() for x in data['split_dict'][split]['data_list']])
test_labels = np.array([x for x in data['split_dict'][split]['label_list']]).reshape(-1,1)

In [ ]:
test_hvs

In [ ]:
test_labels

In [ ]:
from sklearn.metrics import pairwise_distances

dists = pairwise_distances(test_hvs, metric="cosine")

In [ ]:
dists

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

f, ax = plt.subplots(1,1, figsize=(20,20))
sns.heatmap(dists, vmin=-2, vmax=2, cmap="crest", ax=ax)

In [ ]:
print(data["featurizer"].am)

In [ ]:
f, ax = plt.subplots(1,1, figsize=(14,8))
ax.set_xlim([-2,2])
sns.distplot(dists.flatten(), ax=ax)

In [ ]:
result_data.keys()

In [ ]:
model_am = result_data["featurizer"].am
model_am

In [ ]:
import scipy
from scipy.spatial import distance
from numpy.linalg import norm

pos_mask = test_labels == 1
neg_mask = test_labels == 0

class_0_hv = model_am[0].numpy()
class_1_hv = model_am[1].numpy()
# class_0_hv = np.sign(model_am[0].numpy())
# class_1_hv = np.sign(model_am[1].numpy())


# print(test_hvs, model_am[0].numpy())
# distance.cosine(test_hvs[:,0], model_am[0].numpy().reshape(1,-1))
cosine_0 = np.dot(test_hvs,class_0_hv)/(norm(test_hvs, axis=1)*norm(class_0_hv))
cosine_1 = np.dot(test_hvs,class_1_hv)/(norm(test_hvs, axis=1)*norm(class_1_hv))


In [ ]:
print((cosine_1 - cosine_0).shape)
print(pos_mask.shape, neg_mask.shape)


# what happens when we look at the distributions of distances for each class hypervector

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10,10))
f.suptitle("Distance of CASF hypervectors to active class hypervector")
ax.set_xscale("symlog")
sns.histplot(cosine_1[pos_mask.squeeze()], ax=ax)
sns.histplot(cosine_1[neg_mask.squeeze()], ax=ax, color="red")

In [ ]:
f, ax = plt.subplots(1,1, figsize=(10,10))
f.suptitle("Distance of CASF hypervectors to inactive class hypervector")
ax.set_xscale("symlog")
sns.histplot(cosine_0[pos_mask.squeeze()], ax=ax)
sns.histplot(cosine_0[neg_mask.squeeze()], ax=ax, color="red")